# Test MercadoLibre API

Notebook para probar la API oficial de MercadoLibre y ver qué datos nos devuelve.

## Setup
1. Configurar `ML_ACCESS_TOKEN` en tu `.env`
2. Ejecutar las celdas para probar la API
3. Analizar qué campos devuelve vs nuestros modelos


In [ ]:
# Importar librerías
import json
import os
from dotenv import load_dotenv
from src.api.ml_client import MercadoLibreClient
from src.models.database import init_db

# Cargar variables de entorno
load_dotenv()

print("✅ Librerías importadas")
print(f"🔑 ML_ACCESS_TOKEN configurado: {'Sí' if os.getenv('ML_ACCESS_TOKEN') else 'No'}")
print(f"🗄️ DATABASE_URL: {os.getenv('DATABASE_URL', 'No configurado')}")


In [ ]:
# Inicializar cliente de MercadoLibre
client = MercadoLibreClient()
print("✅ Cliente MercadoLibre inicializado")

# Producto de prueba (el aire acondicionado que ya tenemos)
test_item_id = "MLA25265603"
print(f"🎯 Probando con item_id: {test_item_id}")


In [ ]:
# Test 1: Obtener información del producto
print("=== TEST 1: get_product_info ===")
try:
    product_data = client.get_product_info(test_item_id)
    print("✅ Product data obtenido:")
    print(json.dumps(product_data, indent=2, default=str, ensure_ascii=False))
    
    # Verificar qué campos tenemos vs nuestros modelos
    print("\n🔍 Campos disponibles en la API:")
    for key, value in product_data.items():
        print(f"  {key}: {type(value).__name__} = {value}")
        
except Exception as e:
    print(f"❌ Error obteniendo product info: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 2: Obtener reviews del producto
print("=== TEST 2: get_product_reviews ===")
try:
    reviews_data = client.get_product_reviews(test_item_id, limit=3, offset=0)
    print("✅ Reviews data obtenido:")
    print(json.dumps(reviews_data, indent=2, default=str, ensure_ascii=False))
    
    # Analizar estructura de reviews
    if 'reviews' in reviews_data and reviews_data['reviews']:
        print("\n🔍 Estructura de un review:")
        first_review = reviews_data['reviews'][0]
        for key, value in first_review.items():
            print(f"  {key}: {type(value).__name__} = {value}")
    
    # Verificar paginación
    if 'paging' in reviews_data:
        print("\n📄 Info de paginación:")
        print(json.dumps(reviews_data['paging'], indent=2, default=str))
        
except Exception as e:
    print(f"❌ Error obteniendo reviews: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 3: Probar ReviewCacheService
print("=== TEST 3: ReviewCacheService ===")
try:
    from src.services.review_scraper import ReviewCacheService
    from src.models.database import get_session
    
    # Inicializar DB si no existe
    init_db()
    
    svc = ReviewCacheService(client)
    
    with get_session() as db:
        print("\n📦 Probando get_or_fetch_product...")
        product = svc.get_or_fetch_product(db, test_item_id)
        print(f"✅ Producto obtenido: {product.id} - {product.title}")
        print(f"   Marca: '{product.marca}'")
        print(f"   Modelo: '{product.modelo}'")
        print(f"   Características: {product.caracteristicas}")
        
        print("\n📝 Probando fetch_and_store_reviews...")
        reviews = svc.fetch_and_store_reviews(db, test_item_id, limit=2, offset=0)
        print(f"✅ {len(reviews)} reviews guardadas")
        
        if reviews:
            first_review = reviews[0]
            print(f"   Primer review: {first_review.id} - Rating: {first_review.rate}/5")
            print(f"   Source: '{first_review.source}'")
            print(f"   API Review ID: '{first_review.api_review_id}'")
            
except Exception as e:
    print(f"❌ Error probando ReviewCacheService: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Test 4: Comparar con scrape_final.py
print("=== TEST 4: Comparación con scrape_final ===")
print("\n📊 Resumen de diferencias:")
print("\n🔵 API Oficial (ml_client.py):")
print("   ✅ Requiere token")
print("   ✅ Rate limits oficiales")
print("   ✅ Datos estructurados")
print("   ❌ Limitado por permisos del token")

print("\n🟢 API Noindex (scrape_final.py):")
print("   ✅ No requiere token")
print("   ✅ Más permisivo")
print("   ✅ Más reviews por request")
print("   ❌ Puede cambiar sin aviso")

print("\n💡 Recomendación:")
print("   - Usar API oficial para datos básicos y confiabilidad")
print("   - Usar API noindex para obtener más reviews")
print("   - Mantener ambos modos según el caso de uso")


## Próximos pasos

1. **Si obtuviste el token**: Ejecuta todas las celdas y analiza los resultados
2. **Si no tienes token**: La API oficial fallará, pero podrás ver la estructura
3. **Comparar datos**: Ver qué campos devuelve cada API
4. **Decidir estrategia**: Cómo mapear los campos a nuestros modelos

### Para obtener el token:
1. Ir a https://developers.mercadolibre.com.ar/
2. Crear una aplicación
3. Obtener el Access Token
4. Agregarlo a tu `.env` como `ML_ACCESS_TOKEN=tu_token_aqui`
